In [0]:
import requests
import pandas as pd
import numpy as np

# API settings
base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/debt_to_penny"
page_size = 100
page_number = 1
all_data = []

# Pagination loop
while True:
    params = {"page[number]": page_number, "page[size]": page_size}
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f"Error: Status {response.status_code}")
        break

    data = response.json()
    records = data.get("data", [])

    if not records:
        print("End of Data.")
        break

    all_data.extend(records)
    print(f"Fetched page {page_number}, records: {len(records)}")
    page_number += 1

# Convert to DataFrame
df_pd = pd.DataFrame(all_data)

# Rename and cast columns
df_pd = df_pd.rename(columns={
    "record_date": "record_date",
    "debt_held_public_amt": "debt_held_public_amt",
    "intragov_hold_amt": "intragov_hold_amt",
    "tot_pub_debt_out_amt": "tot_pub_debt_out_amt",
    "src_line_nbr": "src_line_nbr",
    "record_fiscal_year": "record_fiscal_year",
    "record_fiscal_quarter": "record_fiscal_quarter",
    "record_calendar_year": "record_calendar_year",
    "record_calendar_quarter": "record_calendar_quarter",
    "record_calendar_month": "record_calendar_month",
    "record_calendar_day": "record_calendar_day"
})

# Handle numeric and null values
decimal_cols = ["debt_held_public_amt", "intragov_hold_amt", "tot_pub_debt_out_amt"]
int_cols = ["src_line_nbr", "record_fiscal_year", "record_fiscal_quarter", "record_calendar_year", "record_calendar_quarter"]

for col in decimal_cols:
    df_pd[col] = pd.to_numeric(df_pd[col], errors='coerce').replace({np.nan: None})

for col in int_cols:
    df_pd[col] = pd.to_numeric(df_pd[col], errors='coerce').astype('Int64')

# Convert to Spark DataFrame
df_spark = spark.createDataFrame(df_pd)

# Write to Delta Lake table in Databricks 'default' database
df_spark.write.format("delta").mode("overwrite").saveAsTable("default.NationalDebtDataAPI")